## Automating the job of an instagram page admin

### Downloads


In [1]:
%%capture
!apt-get update
!apt-get install chromium-chromedriver

In [2]:
!pip install -U -q git+https://github.com/huggingface/transformers accelerate bitsandbytes
!pip install -q selenium

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


### Imports

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from transformers import BitsAndBytesConfig
from PIL import Image
import requests
import torch
import time

from google.colab import files
import io

### Put your image here:

In [60]:
uploaded = files.upload()

Saving 5a8.jpg to 5a8.jpg


### Process the image

In [62]:
file_name = next(iter(uploaded))

img = Image.open(io.BytesIO(uploaded[file_name]))

# Save the image
img_path = '/content/' + file_name
img.save(img_path)

# Assign the image to a variable
image = img

### Time to load the model
this will take some time so take a break 😴

In [56]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", quantization_config=quantization_config, device_map="auto")

preprocessor_config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

#### It is important to prompt the model wth a specific format, which is:
```bash
USER: [INST]<image> <prompt>[/INST
```

the `<prompt>` item contains the actual prompt which is being passed to the AI along with the image

In [63]:
prompt = """[INST] <image> You are a professional Instagram content creator with a million followers. Your task is to help users create engaging posts by suggesting catchy captions and image edits.
For each image, provide a single caption as a string, followed by a dictionary of suggested edits in the format {'edit_type': value}, where 'edit_type' is one of the following: 'Brightness', 'Contrast', 'Fade', 'Saturation', 'Temperature', or 'Vignette', and 'value' is an integer between -1 and 1.
Please provide a single concise output focused on the task at hand. No explanations or justifications are needed.
Let's get started!

Example Response:
"Sunset vibes" {'Brightness': 0.5, 'Contrast': 0.2, 'Fade': -0.3, 'Saturation': 0.8, 'Temperature': 0.1, 'Vignette': 0.4}
[/INST]"""

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

### this is the output the AI gave:

> now we will use this as an input for our instagram post

In [64]:
output = model.generate(**inputs, max_new_tokens=200)

out_string = processor.decode(output[0], skip_special_tokens=True)
out_string

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'[INST]   You are a professional Instagram content creator with a million followers. Your task is to help users create engaging posts by suggesting catchy captions and image edits.\nFor each image, provide a single caption as a string, followed by a dictionary of suggested edits in the format {\'edit_type\': value}, where \'edit_type\' is one of the following: \'Brightness\', \'Contrast\', \'Fade\', \'Saturation\', \'Temperature\', or \'Vignette\', and \'value\' is an integer between -1 and 1.\nPlease provide a single concise output focused on the task at hand. No explanations or justifications are needed.\nLet\'s get started!\n\nExample Response:\n"Sunset vibes" {\'Brightness\': 0.5, \'Contrast\': 0.2, \'Fade\': -0.3, \'Saturation\': 0.8, \'Temperature\': 0.1, \'Vignette\': 0.4}\n[/INST] "Pepe the astronaut" {\'Brightness\': 0.5, \'Contrast\': 0.2, \'Fade\': -0.3, \'Saturation\': 0.8, \'Temperature\': 0.1, \'Vignette\': 0.4} '

### String processing

In [65]:
# i have written better code than this but this works, so shh
def convert_to_dict(input_string):
    inner_string = input_string.strip(" ")
    inner_string = inner_string.strip('}')
    inner_string = inner_string.strip('{')

    # Split into key-value pairs
    pairs = [pair.strip() for pair in inner_string.split(',')]

    result_dict = {}
    for pair in pairs:
        key, value = [part.strip("'") for part in pair.split(': ')]
        value = float(value)
        result_dict[key] = value

    return result_dict

def process(input_string):

    inst_end_index = input_string.index("[/INST]")
    output_part = input_string[inst_end_index + len("[/INST]"):].strip()

    x = output_part.rfind('"')

    caption = output_part[:x].strip().strip('"')
    edits_string = output_part[x + 1:]

    edits_dict = convert_to_dict(edits_string)

    return caption, edits_dict

In [80]:
caption, filter_values = process(out_string)

### lezzgettit instagram dot com 💪💪💪

In [84]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("start-maximized");
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()


In [85]:
driver.get("http://instagram.com")

### Enter ID Password here, I promise I wont steal it :P

In [86]:
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//input[@name="username"]')))

idm = driver.find_element(By.XPATH, '//input[@name="username"]')
passw = driver.find_element(By.XPATH,'//input[@name="password"]')


# write your user name and password to log and pas areas.
loginid = ""  #@param {type:"string"}
password = "" #@param {type:"string"}

idm.send_keys(loginid)
passw.send_keys(password)
time.sleep(2)
#driver.find_element(By.XPATH,'//button[contains(@type,"submit")]').click()

### Rest of the posting happens now

In [87]:
button = driver.find_element(By.XPATH, '//button[contains(@type,"submit")]')
driver.execute_script("arguments[0].click();", button)


errors are a good thing at times

In [88]:
# going thru the save login info, remember me bs
try:
    driver.find_element(By.XPATH, '//button[text()="Not Now"]').click()
except: #NoSuchElementException
    print("not found, good only, move along 🚶‍♂️‍➡️")


not found, good only, move along 🚶‍♂️‍➡️


In [89]:
#find posting button
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="New post"]')))
driver.find_element(By.XPATH, '//*[@aria-label="New post"]').click()

In [90]:
time.sleep(2)
file_input = driver.find_element(By.XPATH, "//input[@type='file']")

file_input.send_keys(img_path) # change to uploaded file

# wait for upload
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[contains(text(),"Next")]')))
driver.find_element(By.XPATH, '//div[contains(text(),"Next")]').click()

In [91]:
# this is where we do the filter magic
time.sleep(2)
driver.find_element(By.XPATH, '//span[contains(text(),"Adjustments")]').click()

filters = ["Brightness", "Contrast", "Fade", "Saturation", "Temperature", "Vignette"]

# sending keys was very slow so interacting through javascript events instead
for fil, value in filter_values.items():
    ranger = driver.find_element(By.XPATH, f"//span[text()='{fil}']/ancestor::div[@role='presentation']//input[@type='range']")
    new_value = value * 100  # Multiply the value by 100 cos model output is between 0-1
    driver.execute_script("arguments[0].value = arguments[1]; arguments[0].dispatchEvent(new Event('input'));", ranger, new_value)


In [92]:
#neeext
driver.find_element(By.XPATH, '//div[contains(text(),"Next")]').click()

In [93]:
#finally putting the caption
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Write a caption...']")))
post_caption = driver.find_element(By.XPATH,"//div[@aria-label='Write a caption...']")
post_caption.send_keys(caption)

In [94]:
time.sleep(2)
driver.find_element(By.XPATH, '//div[contains(text(),"Share")]').click()

In [95]:
WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element((By.XPATH, "//span[text()='Your post has been shared.']"), "Your post has been shared."))
driver.find_element(By.XPATH, '//*[@aria-label="Close"]').click()

### and donezo! the post has been shared :D